In [ ]:
import os
from dotenv import load_dotenv 
import json
import psycopg2
from psycopg2.extras import Json
from azure.storage.blob import BlobServiceClient
from langchain_openai import OpenAIEmbeddings

In [ ]:
# Check 1: Connection to Azure Blob Storage

# Load environment variables from .env file
load_dotenv() 

# Load Azure connection string and container details from environment variables
azure_connection_string = os.getenv("AZURE_CONNECTION_STRING")
container_name = os.getenv("ADLS_CONTAINER_NAME")

# Initialize BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(azure_connection_string)
container_client = blob_service_client.get_container_client(container_name)

# Function to list blobs in a directory
def list_blobs_in_directory(directory_name):
    blob_list = container_client.list_blobs(name_starts_with=directory_name)
    return [blob.name for blob in blob_list if '.' in blob.name]  # Only include blobs with a file extension

# List the blobs in both directories 
counsel_chat_blobs = list_blobs_in_directory("counsel_chat_data")
mental_health_blobs = list_blobs_in_directory("mentalhealth_data")

# Function to get blob size
def get_blob_size(blob_name):
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    properties = blob_client.get_blob_properties()
    size_in_bytes = properties.size
    size_in_mb = size_in_bytes / (1024 * 1024)  # Convert bytes to MB
    return size_in_mb  # Return the size of the blob in MB

# Print out the blob names and their sizes in MB
print("Counsel Chat Files and Sizes:")
for blob_name in counsel_chat_blobs:
    if '.' in blob_name:  # Ensure we're only printing actual files, not directories
        blob_size_mb = get_blob_size(blob_name)
        print(f"File: {blob_name}, Size: {blob_size_mb:.2f} MB")

print("\nMental Health Files and Sizes:")
for blob_name in mental_health_blobs:
    if '.' in blob_name:  # Ensure we're only printing actual files, not directories
        blob_size_mb = get_blob_size(blob_name)
        print(f"File: {blob_name}, Size: {blob_size_mb:.2f} MB")

In [ ]:
# Check 2: Connection to database
# Load environment variables from .env file
load_dotenv() 
# Get PostgreSQL connection details from environment variables
username = os.getenv('PG_ADMIN_USERNAME')  # Get the server admin login name stored in .env file
password = os.getenv('PG_ADMIN_PASSWORD')  # Get the server password stored in .env file
host = os.getenv('PG_SERVER_NAME')         # Get the server name stored in .env file
port = os.getenv('POSTGRES_PORT')          # Get the Standard PostgreSQL port stored in .env file
database = os.getenv('PG_DATABASE')        # Get the database name stored in .env file

# The PostgreSQL connection string 
POSTGRESQL_CONNECTION = f"postgresql://{username}:{password}@{host}:{port}/{database}"

# Connect to the PostgreSQL database
def connect_to_db():
    connection = psycopg2.connect(POSTGRESQL_CONNECTION)
    return connection

# Test the connection
try:
    conn = connect_to_db()
    cursor = conn.cursor()
    print("Nice! Connection successful!")
    cursor.close()
    conn.close()
except Exception as e:
    print(f"Error: {e}")